In [1]:
%matplotlib notebook
# imports
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle 
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion

from IPython.display import HTML

# matplotlib.use('TkAgg')

In [2]:

# basic config and variables
_VERSION = 'v1.0-mini'
_DATAROOT = '/home/jovyan/data'

nusc = NuScenes(version=_VERSION, dataroot=_DATAROOT, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.523 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
def get_pcd_data(nusc_filepath: str):
    radar_point_cloud = RadarPointCloud.from_file(nusc_filepath)
    points =  radar_point_cloud.points
    x = points[0]
    y = points[1]
    vx_comp = points[8]
    vy_comp = points[9]

    return {
        'file': nusc_filepath,
        'x': x,
        'y': y,
        'vx_comp': vx_comp,
        'vy_comp': vy_comp,
        'v_comp': (vx_comp ** 2 + vy_comp ** 2) ** 0.5,
        'radar_point_cloud': radar_point_cloud
    }

def extract_channel_from_file(channel: str):
    filename = nusc.get('sample_data', channel)['filename']
    filename = f'{_DATAROOT}/{filename}'
    return get_pcd_data(filename)


def extract_samples_from_scene(scene: dict):
    """extract all the samples related to the given scene."""
    first_sample_token = scene['first_sample_token']
    last_sample_token = scene['last_sample_token']
    samples = list()

    fst = nusc.get('sample', first_sample_token)
    next_token = fst['next']
    while True:
        current = nusc.get('sample', next_token)
        samples.append(current)
        next_token = current['next']
        if next_token == last_sample_token:
            return samples


def convert_binary_data_to_coordinates_and_velocity(sample: dict):
    data = sample['data']
    return {
     'RADAR_FRONT' : extract_channel_from_file(data['RADAR_FRONT']),
     'RADAR_FRONT_LEFT' : extract_channel_from_file(data['RADAR_FRONT_LEFT']),
     'RADAR_FRONT_RIGHT' : extract_channel_from_file(data['RADAR_FRONT_RIGHT']),
     'RADAR_BACK_LEFT' : extract_channel_from_file(data['RADAR_BACK_LEFT']),
     'RADAR_BACK_RIGHT' : extract_channel_from_file(data['RADAR_BACK_RIGHT']),
     'data': data
    }

sc = nusc.scene[0]
samples_from_scene = extract_samples_from_scene(sc)
scene_in_sample_data = [convert_binary_data_to_coordinates_and_velocity(sc) for sc in samples_from_scene]


In [4]:
def new_method():
    """makes a biggo matrix containing all infos about 2 a)."""
    channels = [
        'RADAR_FRONT',
        'RADAR_FRONT_LEFT',
        'RADAR_FRONT_RIGHT',
        'RADAR_BACK_LEFT',
        'RADAR_BACK_RIGHT',
    ]
    samples = scene_in_sample_data
    scene_points = list()
    for sample in samples:
        x, y, z, vx_comp, vy_comp, pointclouds = list(), list(), list() ,list() ,list(), list()
        ego_pose_coords = []
        for channel in channels:
            pc = sample[channel]['radar_point_cloud']
            radar_token = sample['data'][channel]
            current_radar = nusc.get('sample_data', radar_token)
            ego_pose = nusc.get('ego_pose', current_radar['ego_pose_token'])
            calibrated_sensor = nusc.get('calibrated_sensor', current_radar['calibrated_sensor_token'])
            sensor_to_car = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation'], inverse=False))
            car_to_world = transform_matrix(ego_pose['translation'], Quaternion(ego_pose['rotation'], inverse=False))

            sensor_to_world = car_to_world @ sensor_to_car

            pc.transform(sensor_to_world)

            pointclouds.append(pc)
            
            ego_pose_coords = ego_pose['translation']

            # combine radar
            
            for i in range(pc.points.shape[1]):
                x.append(pc.points[0][i])
                y.append(pc.points[1][i])
                z.append(pc.points[2][i]) # redundant?
                vx_comp.append(pc.points[7][i])
                vy_comp.append(pc.points[8][i])
        scene_points.append([
            np.asarray(x),
            np.asarray(y),
            np.asarray(z),
            np.asarray(vx_comp),
            np.asarray(vy_comp),
            np.asarray(pointclouds),
            np.asarray(ego_pose_coords)
        ])

    return np.asarray(scene_points, dtype=object)


result = new_method()

In [5]:
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

def update(i):
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(900, 1300)
    plt.xlim(300, 550)


    plt.scatter(x, y, s=1)
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')

    plt.plot()


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()

HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

In [6]:
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

def set_color(x, y) -> [str]:
    t = [v[0] < 0 or v[1] < 0 for v in zip(x,y)]
    res = list()
    for b in t:
        if b:
            res.append('aquamarine')
        else:
            res.append('magenta')
    return res


def get_point_coords_from_v_comp(vx_comp, vy_comp, ego_x, ego_y):
    distance_x = vx_comp * 2 # 2 Hz
    distance_y = vy_comp * 2 # 2 Hz
    return (distance_x + ego_x, distance_y + ego_y)
    

def update(i):
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]
    
    new_p = get_point_coords_from_v_comp(vx,vy,x, y)
    
    nx = new_p[0]
    ny = new_p[1]

    plt.cla()
    plt.ylim(900, 1300)
    plt.xlim(300, 550)


    plt.scatter(x, y, s=1, color=set_color(vx, vy))
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    
    for i in range(len(x)):
        plt.arrow(x[i], y[i], nx[i] - x[i], ny[i] - y[i])
    
    plt.plot()


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()

HTML(ani.to_html5_video())

In [7]:
scene_anns = [s['anns'] for s in samples_from_scene]


In [8]:
class Vehicle:
    token = "None"
    x = None
    y = None
    w = None
    h = None
    angle = 0.0
    
    def __init__(self, annotation: dict):
        self.token = annotation['token']
        sample_translation = annotation['translation']
        self.x = sample_translation[0]
        self.y = sample_translation[1]
        sample_size = annotation['size']
        self.h = sample_size[1]
        self.w = sample_size[0]


    def get_left_corner(self):
        x_left = self.x - (self.w / 2)
        y_left = self.y - (self.h / 2)
        return (x_left, y_left)
    
    def get_trans(self):
        return (self.x, self.y)
    
    def __repr__(self):
        return f'TOKEN:{self.token}\tTRANSL:{self.get_trans()}\tLEFT_CORNER:{self.get_left_corner()}'


def get_vehicles_from_sample(ann_ids: [str]):
    anns = [nusc.get('sample_annotation', a_id) for a_id in ann_ids]
    anns = [v for v in anns if 'vehicle.' in v['category_name']]
    veh = [Vehicle(a) for a in anns]
    return veh

get_vehicles_from_sample(scene_anns[0])

[TOKEN:f0cbd9dbafd74e20bcf6dd0357c97f59	TRANSL:(353.776, 1132.363)	LEFT_CORNER:(352.7705, 1130.0465000000002),
 TOKEN:86214ec54d034a839ee1f400719d49b2	TRANSL:(372.664, 1129.247)	LEFT_CORNER:(372.3195, 1128.362),
 TOKEN:8161dbd026154299827eb67bf053ecc3	TRANSL:(411.119, 1205.87)	LEFT_CORNER:(410.20050000000003, 1203.7099999999998),
 TOKEN:a286c9633fa34da5b978758f348996b0	TRANSL:(392.945, 1148.426)	LEFT_CORNER:(392.091, 1146.4209999999998),
 TOKEN:f3721bdfd7ee4fd2a4f94874286df471	TRANSL:(409.998, 1164.084)	LEFT_CORNER:(408.5595, 1158.9835),
 TOKEN:7e077a909fda478caedde939b6114202	TRANSL:(392.077, 1104.169)	LEFT_CORNER:(391.0095, 1101.691),
 TOKEN:91937fcf68224fbd97ec4de10ec4e447	TRANSL:(423.565, 1237.894)	LEFT_CORNER:(422.1105, 1234.44),
 TOKEN:148b3b0a63434e8d8dd0c2b56dcc4fa5	TRANSL:(391.841, 1138.065)	LEFT_CORNER:(390.9175, 1136.0075000000002),
 TOKEN:6bd43869a39842d5860364b5274d011f	TRANSL:(360.932, 1129.086)	LEFT_CORNER:(359.96250000000003, 1126.6765),
 TOKEN:787ce4c9d8e840d399c390e25

In [9]:
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

count = 0
last = None 


def update(i):
    global count
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(1000, 1300)
    plt.xlim(300, 500)
    vehicles = get_vehicles_from_sample(scene_anns[count])
    veh_coords = [v.get_trans() for v in vehicles]
    count += 1
    veh_coords_x = [x[0] for x in veh_coords]
    veh_coords_y = [x[1] for x in veh_coords]

#     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
#     fig = plt.subplot()
    
    plt.scatter(x, y, s=1)
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    ego_w = 30
    ego_h = 20
    
    plt.gca().add_patch(Rectangle((radar_ego_x - 15, radar_ego_y - 10), 30, 20, color="white", fill=False))
    
    for i in range(len(veh_coords_y)):
        color = 'yellow'
        ego_off_x = radar_ego_x - ego_w / 2
        ego_off_y = radar_ego_y - ego_h / 2
        is_in_x = ego_off_x <= veh_coords_x[i] <= radar_ego_x + ego_w / 2
        is_in_y = ego_off_y <= veh_coords_y[i] <= radar_ego_y + ego_h / 2
        if is_in_x and is_in_y:
            global last
            last = vehicles[i]
            color="magenta"
            print(last)
        plt.gca().add_patch(Rectangle((veh_coords_x[i],veh_coords_y[i]), vehicles[i].w, vehicles[i].h, color=color, fill=False))
    
    plt.plot()

    


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()
HTML(ani.to_html5_video())

TOKEN:3b1c9f82380a4ecdaf43febdfdbcc0a8	TRANSL:(410.007, 1164.069)	LEFT_CORNER:(408.56850000000003, 1158.9685)
TOKEN:a362ccc1c6ac4e0e88703e44cac714d9	TRANSL:(409.982, 1163.956)	LEFT_CORNER:(408.54350000000005, 1158.8555)
TOKEN:7d616ddd17fb4b4ab4aa81ce3403dd94	TRANSL:(409.96, 1163.891)	LEFT_CORNER:(408.5215, 1158.7905)
TOKEN:4fc2c86cf0db4c7290831b39e6adf8d8	TRANSL:(409.938, 1163.826)	LEFT_CORNER:(408.4995, 1158.7255)
TOKEN:1a59783e6dff4c698a0c2ab8cd666cd3	TRANSL:(409.938, 1163.826)	LEFT_CORNER:(408.4995, 1158.7255)
TOKEN:295f8789c3d94ad19b6a0e5b7b6f7632	TRANSL:(409.295, 1117.038)	LEFT_CORNER:(408.873, 1115.9235)
TOKEN:b2715b9de0fe4eb7875ed14694aa9485	TRANSL:(409.323, 1117.058)	LEFT_CORNER:(408.90099999999995, 1115.9435)
TOKEN:a7e1c279fdf04f6a892ee5c4f52bdb63	TRANSL:(409.351, 1117.078)	LEFT_CORNER:(408.929, 1115.9635)
TOKEN:2401a4c146ca4312a917713f99371c8a	TRANSL:(409.379, 1117.098)	LEFT_CORNER:(408.957, 1115.9835)
TOKEN:6b47773ea624459ea8f70883df1494aa	TRANSL:(398.565, 1110.043)	LEFT_COR

In [10]:
_SCENE_LENGTH = 37
# samples_from_scene
def extract_sample_annotations_from_scene(scene: dict):
    """extract all the sample annotations related to the given scene."""
    first_sample_token = scene['first_sample_token']
    last_sample_token = scene['last_sample_token']
    
    sample_annotations = []
    
    fst = nusc.get('sample', first_sample_token)
    current = fst
    
    while True:
        [sample_annotations.append(a) for a in nusc.sample_annotation
            if a['sample_token'] == current['token'] and '' in a['category_name']]
        if current['token'] == last_sample_token:
            return sample_annotations
        current = nusc.get('sample', current['next'])

def walk_annotations(d, collector=[]):
    if d['next'] == '':
        return collector
    collector.append(d)
    return walk_annotations(nusc.get('sample_annotation', d['next']), collector)

_DATA = dict()

# res = walk_annotations(nusc.get('sample_annotation', 'bde622696e974bb8a41e7c681528d36e'))
# print(res)

for ann in extract_sample_annotations_from_scene(sc):
    global _DATA
    if ann['prev'] == '' and 'c3246a1e22a14fcb878aa61e69ae3329' in ann['attribute_tokens']:# and nusc.get('attribute', ann['attribute_tokens'][0])['name'] == 'vehicle.moving':
        _DATA[ann['token']] = len(walk_annotations(ann))

_DATA

{'924ee6ac1fed440a9d9e3720aac635a0': 36,
 '74986f1604f047b6925d409915265bf7': 74,
 'a6f7d4bb60374f868144c5ba4431bf4c': 112,
 '91cb8f15ed4444e99470d43515e50c1d': 150,
 '26fb370c13f844de9d1830f6176ebab6': 181,
 'c5774936c46742ce9ef427016bc3ac6e': 206,
 '3d79d7c4d2734de09299dd8abeab0ad3': 242,
 '59e05c3569054c7fbe6b50d134560f76': 260,
 '9011c733d2f44658a515c2e01a71bc44': 295,
 '04b0d9bec8c04b19abbe76dc0c67a7ab': 330,
 'c620d21632554274905bd6bf66827166': 364,
 '13b1a6e7bfac4d7eb6a1edf9bcb9ad82': 393,
 '84ff661e471645a4ad77e894edcd8939': 419,
 '38412e58d972491582cf21a19ea31a80': 443,
 '5049b74446b34579b2e32e7399a43b97': 443}

In [11]:
'cb5118da1ab342aa947717dc53544259' in nusc.get('sample_annotation', '5049b74446b34579b2e32e7399a43b97')['attribute_tokens'][0]

False

In [12]:
def draw_instance(annotation_id):
    plt.style.use('dark_background')

    ys = list()
    xs = list()
    ego_x = list()
    ego_y = list()

    result_iterator = iter(result)

    count = 0

    FOCUS_VEHICLE = Vehicle(nusc.get('sample_annotation', annotation_id))

    def update(i):
        global count
        try:
            row = next(result_iterator)
        except StopIteration:
            return
        x, y = row[0], row[1]
        vx, vy = row[3], row[4]
        radar_ego_x = row[6][0]
        radar_ego_y = row[6][1]

        plt.cla()
        plt.ylim(1000, 1300)
        plt.xlim(300, 500)
        vehicles = [FOCUS_VEHICLE]
        veh_coords = [v.get_trans() for v in vehicles]
        count += 1
        veh_coords_x = [x[0] for x in veh_coords]
        veh_coords_y = [x[1] for x in veh_coords]

    #     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
    #     fig = plt.subplot()

        plt.scatter(x, y, s=1)
        plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
        for i in range(len(veh_coords_y)):
            plt.gca().add_patch(Rectangle((veh_coords_x[i],veh_coords_y[i]), vehicles[i].w, vehicles[i].h, color="yellow", fill=False))

        plt.plot()




    ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
    # plt.show()

    return HTML(ani.to_html5_video())
    
draw_instance('d02856dbe859476f9635f449d23aa211')


In [13]:
last = nusc.get('sample_annotation', 'd02856dbe859476f9635f449d23aa211')
FOCUS = []
while last['prev'] != '':
    FOCUS.append(last)
    last =  nusc.get('sample_annotation',last['prev'])

FOCUS_VEHICLES = [Vehicle(nusc.get('sample_annotation', f['token'])) for f in FOCUS]

FOCUS_VEHICLES

[TOKEN:d02856dbe859476f9635f449d23aa211	TRANSL:(428.504, 1106.56)	LEFT_CORNER:(427.5505, 1104.1965),
 TOKEN:cad8d5a3f7cc4c55bc44da68369f49da	TRANSL:(427.868, 1105.779)	LEFT_CORNER:(426.9145, 1103.4155),
 TOKEN:1263af5649bd4c6caa33272276a0329c	TRANSL:(427.101, 1105.125)	LEFT_CORNER:(426.1475, 1102.7615),
 TOKEN:2019267b05624caf8cbdf9db842d3cf5	TRANSL:(426.23, 1104.619)	LEFT_CORNER:(425.2765, 1102.2555),
 TOKEN:4e9a65ceca9a4533aed3cfd6ff8eba08	TRANSL:(425.177, 1104.334)	LEFT_CORNER:(424.2235, 1101.9705000000001),
 TOKEN:dfcd70bb4efa4ae29a99cf19d3d0cfa0	TRANSL:(423.942, 1104.199)	LEFT_CORNER:(422.9885, 1101.8355000000001),
 TOKEN:c2721bb7f86f48e58800f40a3b5ad613	TRANSL:(422.729, 1104.393)	LEFT_CORNER:(421.77549999999997, 1102.0295),
 TOKEN:97a12a3973934204bc927bf2c6328e16	TRANSL:(421.476, 1104.531)	LEFT_CORNER:(420.5225, 1102.1675),
 TOKEN:8633be668e3445b19f26465ff6bdf5a4	TRANSL:(420.201, 1104.947)	LEFT_CORNER:(419.2475, 1102.5835),
 TOKEN:cdfd8e85d17942f8820c2f234239b874	TRANSL:(419.123,

In [14]:
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

count = 0
last = None 


def update(i):
    global count
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(1000, 1300)
    plt.xlim(300, 500)
    vehicles = FOCUS_VEHICLES
    veh_coords = [v.get_trans() for v in vehicles]
    veh_coords_x = [x[0] for x in veh_coords]
    veh_coords_y = [x[1] for x in veh_coords]
    veh_coords_x.reverse()
    veh_coords_y.reverse()
    
    veh_coords_x = np.asarray(veh_coords_x)
    veh_coords_y = np.asarray(veh_coords_y)

#     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
#     fig = plt.subplot()
    
    plt.scatter(x, y, s=1)
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    ego_w = 30
    ego_h = 20
    lens = 3
    i = count
#     plt.gca().add_patch(Rectangle((radar_ego_x - 15, radar_ego_y - 10), 30, 20, color="white", fill=False))
    plt.gca().add_patch(Rectangle((veh_coords_x[i],veh_coords_y[i]), vehicles[i].w * lens, vehicles[i].h * lens, color='yellow', fill=False))
    count += 1

    #     for i in range(len(veh_coords_y)):
#         color = 'yellow'
# #         ego_off_x = radar_ego_x - ego_w / 2
# #         ego_off_y = radar_ego_y - ego_h / 2
# #         is_in_x = ego_off_x <= veh_coords_x[i] <= radar_ego_x + ego_w / 2
# #         is_in_y = ego_off_y <= veh_coords_y[i] <= radar_ego_y + ego_h / 2
# #         if is_in_x and is_in_y:
# #             global last
# #             last = vehicles[i]
# #             color="magenta"
# #             print(last)
#         if True:
#             plt.gca().add_patch(Rectangle((veh_coords_x[i],veh_coords_y[i]), vehicles[i].w, vehicles[i].h, color=color, fill=False))
    
    plt.plot()

    


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()
HTML(ani.to_html5_video())

In [21]:
# all radar points in box
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

count = 0
last = None 


def update(i):
    global count
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(1000, 1300)
    plt.xlim(300, 500)
    vehicles = FOCUS_VEHICLES
    veh_coords = [v.get_trans() for v in vehicles]
    veh_coords_x = [x[0] for x in veh_coords]
    veh_coords_y = [x[1] for x in veh_coords]
    veh_coords_x.reverse()
    veh_coords_y.reverse()
    
    veh_coords_x = np.asarray(veh_coords_x)
    veh_coords_y = np.asarray(veh_coords_y)

#     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
#     fig = plt.subplot()
    

    ego_w = 30
    ego_h = 20
    lens = 3
    current_x_min = veh_coords_x[count]
    current_y_min = veh_coords_y[count]
    width = vehicles[count].w * lens
    height = vehicles[count].h * lens
    plt.gca().add_patch(Rectangle((current_x_min,current_y_min), width, height, color='yellow', fill=False))
    count += 1
    coords, new_x, new_y = [],[],[]
    for p in list(zip(x,y)):
        is_in_x = current_x_min <= p[0] <= current_x_min + width
        is_in_y = current_y_min <= p[1] <= current_y_min + height
        if is_in_x and is_in_y:
            coords.append(p)
    
    new_x = [t[0] for t in coords]
    new_y = [t[1] for t in coords]
    plt.scatter(new_x, new_y, s=1)
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    plt.plot()

    


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()
HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

In [20]:
# average points
from statistics import mean
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

count = 0
last = None 


def update(i):
    global count
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(1000, 1300)
    plt.xlim(300, 500)
    vehicles = FOCUS_VEHICLES
    veh_coords = [v.get_trans() for v in vehicles]
    veh_coords_x = [x[0] for x in veh_coords]
    veh_coords_y = [x[1] for x in veh_coords]
    veh_coords_x.reverse()
    veh_coords_y.reverse()
    
    veh_coords_x = np.asarray(veh_coords_x)
    veh_coords_y = np.asarray(veh_coords_y)

#     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
#     fig = plt.subplot()
    

    ego_w = 30
    ego_h = 20
    lens = 3
    current_x_min = veh_coords_x[count]
    current_y_min = veh_coords_y[count]
    width = vehicles[count].w * lens
    height = vehicles[count].h * lens
    plt.gca().add_patch(Rectangle((current_x_min,current_y_min), width, height, color='yellow', fill=False))
    count += 1
    coords, new_x, new_y = [],[],[]
    for p in list(zip(x,y)):
        is_in_x = current_x_min <= p[0] <= current_x_min + width
        is_in_y = current_y_min <= p[1] <= current_y_min + height
        if is_in_x and is_in_y:
            coords.append(p)
    
    new_x = ([t[0] for t in coords])
    new_y = ([t[1] for t in coords])
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    if new_x == [] or new_y == []:
            plt.plot()
            return
    new_x = sum(new_x) / len(new_x)
    new_y = sum(new_y) / len(new_y)
    plt.scatter(new_x, new_y, s=1)
    plt.plot()

    


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()
HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

In [26]:
# average points
from statistics import mean
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

count = 0
last = None 


def update(i):
    global count
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]

    plt.cla()
    plt.ylim(1000, 1300)
    plt.xlim(300, 500)
    vehicles = FOCUS_VEHICLES
    veh_coords = [v.get_trans() for v in vehicles]
    veh_coords_x = [x[0] for x in veh_coords]
    veh_coords_y = [x[1] for x in veh_coords]
    veh_coords_x.reverse()
    veh_coords_y.reverse()
    
    veh_coords_x = np.asarray(veh_coords_x)
    veh_coords_y = np.asarray(veh_coords_y)

#     plt.scatter(veh_coords_x, veh_coords_y, s=20, color="white")
#     fig = plt.subplot()
    

    ego_w = 30
    ego_h = 20
    lens = 3
    current_x_min = veh_coords_x[count]
    current_y_min = veh_coords_y[count]
    width = vehicles[count].w * lens
    height = vehicles[count].h * lens
    plt.gca().add_patch(Rectangle((current_x_min,current_y_min), width, height, color='yellow', fill=False))
    coords, new_x, new_y = [],[],[]
    for p in list(zip(x,y)):
        is_in_x = current_x_min <= p[0] <= current_x_min + width
        is_in_y = current_y_min <= p[1] <= current_y_min + height
        if is_in_x and is_in_y:
            coords.append(p)
    
    new_x = ([t[0] for t in coords])
    new_y = ([t[1] for t in coords])
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    plt.scatter(veh_coords_x[count], veh_coords_y[count], s=50, color='aquamarine')
    if new_x == [] or new_y == []:
            plt.plot()
            return
    new_x = sum(new_x) / len(new_x)
    new_y = sum(new_y) / len(new_y)
    plt.scatter(new_x, new_y, s=20, color='blue')
    plt.plot()
    count += 1

    


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()
HTML(ani.to_html5_video())


<IPython.core.display.Javascript object>